# Welcome!

This is the first notebook I ever published,kind of learning 
The trick that boosted my score significantly was to keep track of the attempt of every user for every question, using this discussion as inspiration.

A state was used during the prediction pahse where also the mean user accuracy and answered (correctly) feature were updated.

I will not disclose my features dataset as this notebook is purely meant for inspiration and not for copying and rerunning.

All steps will be provided of a short explanation, but if you have any questions feel free to ask them in the comments!

In [1]:
import numpy as np
import pandas as pd

import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import roc_auc_score
from multiprocessing import cpu_count
from tqdm.notebook import tqdm

# these imports are used to convert the tree to PNG
from cairosvg import svg2png
from PIL import Image
from io import BytesIO

import gc
import os
import sys

In [2]:
VERSION = 'V1E'
NUM_BOOST_ROUND = 2500
VERBOSE_EVAL = 10
METRICS = ['auc']
N_ROWS = 99271300

def get_index_np():
    return np.arange(N_ROWS) # speeds up dataframe creation

In [3]:
# features are saved as compressed numpy arrays, much more efficient than a pandas DataFrame!
FEATURES = np.load('../input/data-merged/light_train_data.npz',allow_pickle=True)#, )

In [4]:
lagtime_mean= 20441941.566456214
lagtime_mean2= 40370652.30368494
lagtime_mean3= 60008399.99610778
delta_prior_question_elapsed_time_mean= -8.967329025268555

In [5]:
#nbr_answred_correctly_after_explanation
#nbr_answred_incorrectly_after_explanation
given_features = [
    'prior_question_elapsed_time',
]

deduced_features = [
    #'prior_lag_time',
    'lagtime',
    'lagtime2',
    'lagtime3',
    'delta_prior_question_elapsed_time',
     #'roll_prior_elapsed_time',
    #'prior_question_elapsed_time',
    #'roll_prior_lag_time',
    'content_id',
    'task_container_id',
    # user features
    'user_correctness',
    'answered_correctly_user',
    'answered_user',
    'answered_incorrectly_user',
    'user_residual',
    'user_residual_part',
    #'user_residual_after_exp',
    #'user_residual_no_exp',
    #'residual_bundle_elapsed_time',
    #'residual_user_had_explanation',
    
    #Lecture features
    'lecture_seen',
    'lecture_part_seen',
    
    'mean_prior_elapsed_time',
    'mean_prior_had_explanation',
    'all_prior_had_explanation',
    'answered_correctly_after_explanation',
    'answered_incorrectly_after_explanation',
    'answered_correctly_after_no_explanation',
    'answered_incorrectly_after_no_explanation',
    
    # content features
    'question_accuracy',
    'question_nbr_seen',
    'avg_question_elapsed_time',
    'avg_question_had_exlanation',
    #'avg_bundle_elapsed_time',
    #'bundle_size',
    
    # part features
    'part',
    'h_part_accuracy',
    'user_part_correctness',
    'answered_correctly_user_part',
    'answered_user_part',
    'answered_incorrectly_user_part',
    #'hmean_user_part_accuracy',
    
    # other features
    'hmean_user_content_accuracy',
    #'h_residual_user',
    #'h_prior_elapsed_time',
    #'h_prior_had_explanation',
    'attempt',
    'prior_question_had_explanation'
    ,'tags_label'
]

features = given_features + deduced_features

target = 'answered_correctly'

# add categorical features indices
categorical_feature = ['part', 'tags', 'tags_label', 'prior_question_had_explanation','content_id','task_container_id']#,'bundle_size'
categorical_feature_idxs = []
for v in categorical_feature:
    try:
        categorical_feature_idxs.append(features.index(v))
    except:
        pass

# Prediction Preparation

In [6]:
# dataframe with question features used for merging with test_df
#def get_features_questions_df():
#    # create DataFrame of features
#    features_df = pd.DataFrame(index=get_index_np())
#    #['content_id', 'part', 'tags', 'tags_label', 'question_accuracy']
#    for col in tqdm(['content_id', 'part','tags_label','question_accuracy','question_nbr_seen',
#                     'avg_question_elapsed_time','avg_question_had_exlanation']):
#        features_df[col] = FEATURES[col]
#
#    # content features
#    features_questions_df = features_df.groupby('content_id')[[
#        # merge keys
#        'content_id',
#        'part',
#        #'tags',
#        'tags_label',
#        # content
#        'question_accuracy',
#        'question_nbr_seen',
#        'avg_question_elapsed_time',
#        'avg_question_had_exlanation'
#    ]].first().reset_index(drop=True).sort_values('content_id')
#    
#    return features_questions_df
    
#features_questions_df = get_features_questions_df()
#print(f'features_questions_df, rows: {features_questions_df.shape[0]}')
#display(features_questions_df.head())

features_questions_df = pd.read_pickle('../input/data-merged/question_to_merge_v4.pkl') #get_features_questions_df()
print(f'features_questions_df, rows: {features_questions_df.shape[0]}')
display(features_questions_df.head())

features_questions_df, rows: 13523


,question_id,part,tags,tags_label,question_accuracy,question_nbr_seen,avg_question_elapsed_time,avg_question_had_exlanation,part_accuracy,h_part_accuracy
0,0,1,51 131 162 38,981,0.907721,0.940345,19693.703125,0.951326,0.745032,0.818370
1,1,1,131 36 81,307,0.890646,1.007775,19320.742188,0.979454,0.745032,0.811358
2,2,1,131 101 162 92,251,0.554281,6.117078,24574.775391,0.900145,0.745032,0.635655
3,3,1,131 149 162 29,287,0.779437,3.129443,21628.972656,0.966744,0.745032,0.761846
4,4,1,131 5 162 38,317,0.613215,4.323162,22046.859375,0.532298,0.745032,0.672727


This next function is the most important part of this notebook, it creates a state to keep track of attempt, mean user accuracy and the total amount of (correct) questions answered. This takes ~2 minutes to generate.

The state is implemented using dictionaries, as this is more efficient than using a DataFrame.

An example of the state for a user is given below.

In [7]:
def get_state():
    # create DataFrame of features
    features_df = pd.DataFrame(index=get_index_np())
    for col in tqdm(['user_id', 'content_id', 'answered_correctly','part','prior_question_elapsed_time','prior_question_had_explanation']):
        features_df[col] = FEATURES[col]
        
        
        

    prior_question_elapsed_time=features_df[['user_id','prior_question_elapsed_time']].groupby('user_id',as_index=False).tail(1)
    prior_question_elapsed_time['prior_question_elapsed_time']=prior_question_elapsed_time['prior_question_elapsed_time'].astype(int)
    prior_question_elapsed_time['prior_question_elapsed_time'].fillna(23916, inplace=True)
    
    user_prior_question_elapsed_time_dict=dict()
    for cnt,row in tqdm(enumerate(prior_question_elapsed_time.values)):
        user_prior_question_elapsed_time_dict[row[0]] = row[1]
        
    # compute user features over all train data
    mean_user_accuracy = features_df.groupby('user_id')['answered_correctly'].mean().values
    answered_correctly_user = features_df.groupby('user_id')['answered_correctly'].sum().values
    answered_user = features_df.groupby('user_id')['answered_correctly'].count().values
    answered_incorrectly_user=answered_user-answered_correctly_user
    
    features_df['answered_correctly_after_explanation']=features_df['answered_correctly']*features_df['prior_question_had_explanation']
    features_df['answered_incorrectly_after_explanation']=(1-features_df['answered_correctly'])*features_df['prior_question_had_explanation']
    features_df['answered_correctly_after_no_explanation']=features_df['answered_correctly']*(1-features_df['prior_question_had_explanation'])
    features_df['answered_incorrectly_after_no_explanation']=(1-features_df['answered_correctly'])*(1-features_df['prior_question_had_explanation'])
    
    answered_correctly_after_explanation=features_df.groupby('user_id')['answered_correctly_after_explanation'].sum().values
    answered_incorrectly_after_explanation=features_df.groupby('user_id')['answered_incorrectly_after_explanation'].sum().values
    answered_correctly_after_no_explanation=features_df.groupby('user_id')['answered_correctly_after_no_explanation'].sum().values
    answered_incorrectly_after_no_explanation=features_df.groupby('user_id')['answered_incorrectly_after_no_explanation'].sum().values
    
    
    last_prior_data=pd.read_pickle('../input/data-merged/last_prior_data.pkl')
    lecture_seen=pd.read_pickle('../input/data-merged/lecture_seen.pkl')
    lecture_part_seen=pd.read_pickle('../input/data-merged/lecture_part_seen.pkl')
    
    residual_data=pd.read_pickle('../input/data-merged/residual_data.pkl')
    residual_data_part=pd.read_pickle('../input/data-merged/residual_data_part.pkl')
    #mean_prior_elapsed_time=features_df.groupby('user_id')['prior_question_elapsed_time'].mean().values
    #nbr_prior_had_explanation=features_df.groupby('user_id')['prior_question_had_explanation'].sum().values
    #mean_prior_had_explanation=nbr_prior_had_explanation/answered_user
    
    
    # fill dictionary with default values
    state = dict()
    for user_id in features_df['user_id'].unique():
        state[user_id] = {'user_residual_part':{},'lecture_part_seen':{},'lecture_seen': 0}
    total = len(state.keys())
        
    # add user content attempts
    user_content = features_df.groupby('user_id')['content_id'].apply(np.array).apply(np.sort).apply(np.unique)
    user_attempts = features_df.groupby(['user_id', 'content_id'])['content_id'].count().astype(np.uint8).groupby('user_id').apply(np.array).values
    user_attempts -= 1
    
    # add user part data
    user_part = features_df.groupby('user_id')['part'].apply(np.array).apply(np.sort).apply(np.unique)
    mean_user_part_accuracy = features_df.groupby(['user_id','part'])['answered_correctly'].mean().groupby('user_id').apply(np.array).values
    answered_correctly_user_part = features_df.groupby(['user_id','part'])['answered_correctly'].sum().groupby('user_id').apply(np.array).values
    answered_user_part = features_df.groupby(['user_id','part'])['answered_correctly'].count().groupby('user_id').apply(np.array).values
    answered_incorrectly_user_part=answered_user_part-answered_correctly_user_part
    
    del features_df
    gc.collect()
    
    for user_id, part, user_part_accuracy,ans_cor_user_part,ans_user_part,ans_inc_user_part in tqdm(zip(state.keys(), user_part, mean_user_part_accuracy,answered_correctly_user_part,answered_user_part,answered_incorrectly_user_part),total=total):
        state[user_id]['mean_user_part_accuracy'] = dict(zip(part, user_part_accuracy))
        state[user_id]['answered_correctly_user_part'] = dict(zip(part, ans_cor_user_part))
        state[user_id]['answered_user_part'] = dict(zip(part, ans_user_part))
        state[user_id]['answered_incorrectly_user_part'] = dict(zip(part, ans_inc_user_part))
    
    del user_part,mean_user_part_accuracy,answered_correctly_user_part,answered_user_part,answered_incorrectly_user_part
    gc.collect()
    
    
    for user_id, content, attempt in tqdm(zip(state.keys(), user_content, user_attempts),total=total):
        state[user_id]['user_content_attempts'] = dict(zip(content, attempt))
    
    del user_content, user_attempts
    gc.collect()
        
    for cnt,row in tqdm(enumerate(residual_data.values)):
        state[row[0]]['user_residual']=row[1]
        
    #'user_id','last_task_id','nbr_question_task','prior_question_nbr','mean_prior_elapsed_time','mean_prior_had_explanation','all_prior_had_explanation'    
    for cnt,row in tqdm(enumerate(last_prior_data.values)):
        state[row[0]]['nbr_question_task']=row[2]
        state[row[0]]['prior_question_nbr']=row[3]
        state[row[0]]['mean_prior_elapsed_time'] = row[4]
        state[row[0]]['mean_prior_had_explanation'] = row[5]
        state[row[0]]['all_prior_had_explanation'] = row[6]
    
    #'user_id','last_task_id','nbr_question_task','prior_question_nbr','mean_prior_elapsed_time','mean_prior_had_explanation','all_prior_had_explanation'    
    for cnt,row in tqdm(enumerate(lecture_seen.values)):
        state[row[0]]['lecture_seen']=row[1]
    
    for cnt,row in tqdm(enumerate(residual_data_part.values)):
        state[row[0]]['user_residual_part'][row[1]]=row[2]
    
    for cnt,row in tqdm(enumerate(lecture_part_seen.values)):
        state[row[0]]['lecture_part_seen'][row[1]]=row[2]
        
    
    del lecture_seen,residual_data,last_prior_data,residual_data_part,lecture_part_seen
    gc.collect()
    
    
    for idx, user_id in enumerate(state.keys()):
        state[user_id]['mean_user_accuracy'] = mean_user_accuracy[idx]
        state[user_id]['answered_correctly_user'] = answered_correctly_user[idx]
        state[user_id]['answered_user'] = answered_user[idx]
        state[user_id]['answered_incorrectly_user'] = answered_incorrectly_user[idx]
        
        state[user_id]['answered_correctly_after_explanation'] = answered_correctly_after_explanation[idx]
        state[user_id]['answered_incorrectly_after_explanation'] = answered_incorrectly_after_explanation[idx]
        state[user_id]['answered_correctly_after_no_explanation'] = answered_correctly_after_no_explanation[idx]
        state[user_id]['answered_incorrectly_after_no_explanation'] = answered_incorrectly_after_no_explanation[idx]
    
    del mean_user_accuracy,answered_correctly_user,answered_user,answered_incorrectly_user,answered_correctly_after_explanation,answered_incorrectly_after_explanation,answered_correctly_after_no_explanation,answered_incorrectly_after_no_explanation
    gc.collect()
    
    
    max_timestamp_u=pd.read_csv('../input/data-merged/max_timestamp_u.csv')
    max_timestamp_u2=pd.read_csv('../input/data-merged/max_timestamp_u2.csv')
    max_timestamp_u3=pd.read_csv('../input/data-merged/max_timestamp_u3.csv')
    max_timestamp_u['max_time_stamp']=max_timestamp_u['max_time_stamp'].astype(float)
    
    max_timestamp_u2['max_time_stamp2'].fillna(lagtime_mean2, inplace=True)
    max_timestamp_u3['max_time_stamp3'].fillna(lagtime_mean3, inplace=True)
    
    
    
    max_timestamp_u_dict = dict()
    for cnt,row in tqdm(enumerate(max_timestamp_u.values)):
        max_timestamp_u_dict[row[1]] = row[2]
        
    max_timestamp_u_dict2 = dict()
    for cnt,row in tqdm(enumerate(max_timestamp_u2.values)):
        max_timestamp_u_dict2[row[1]] = row[2]
        
    max_timestamp_u_dict3 = dict()
    for cnt,row in tqdm(enumerate(max_timestamp_u3.values)):
        max_timestamp_u_dict3[row[1]] = row[2]
    
    
    
    del max_timestamp_u,max_timestamp_u2,max_timestamp_u3
    
    return state,max_timestamp_u_dict,max_timestamp_u_dict2,max_timestamp_u_dict3,user_prior_question_elapsed_time_dict
    

state,max_timestamp_u_dict,max_timestamp_u_dict2,max_timestamp_u_dict3,user_prior_question_elapsed_time_dict = get_state()
gc.collect()
print('Example of the state for user 2746, attempt counting starts at 0 as the pandas cumcount function is used to create the attempt feature')
display(state[2746])


Example of the state for user 2746, attempt counting starts at 0 as the pandas cumcount function is used to create the attempt feature


{'user_residual_part': {2.0: -0.10299227894746332, 5.0: -0.5647877542860366},
 'lecture_part_seen': {2.0: 1.0, 5.0: 0.0},
 'lecture_seen': 1.0,
 'mean_user_part_accuracy': {2: 0.6470588235294118, 5: 0.0},
 'answered_correctly_user_part': {2: 11.0, 5: 0.0},
 'answered_user_part': {2: 17, 5: 2},
 'answered_incorrectly_user_part': {2: 6.0, 5: 2.0},
 'user_content_attempts': {236: 0,
  294: 0,
  297: 1,
  382: 0,
  404: 0,
  405: 0,
  484: 0,
  531: 0,
  714: 0,
  758: 0,
  775: 1,
  873: 0,
  1014: 0,
  1295: 0,
  5273: 0,
  5976: 0,
  10684: 0},
 'user_residual': -0.15160232898310258,
 'nbr_question_task': 1.0,
 'prior_question_nbr': 18.0,
 'mean_prior_elapsed_time': 18055.555555555555,
 'mean_prior_had_explanation': 0.6111111111111112,
 'all_prior_had_explanation': 11.0,
 'mean_user_accuracy': 0.5789473684210527,
 'answered_correctly_user': 11.0,
 'answered_user': 19,
 'answered_incorrectly_user': 8.0,
 'answered_correctly_after_explanation': 7.0,
 'answered_incorrectly_after_explanatio

In [8]:
def get_user_data(state, test_df):
    # updated data
    attempt, mean_user_accuracy, answered_correctly_user, answered_user,answered_incorrectly_user,user_residual,lecture_seen = [], [], [], [], [], [], []
    mean_prior_elapsed_time,all_prior_had_explanation,mean_prior_had_explanation=[],[],[]
    mean_user_part_accuracy, answered_correctly_user_part, answered_user_part,answered_incorrectly_user_part,user_residual_part,lecture_part_seen = [], [], [], [], [], []
    answered_correctly_after_explanation,answered_incorrectly_after_explanation,answered_correctly_after_no_explanation,answered_incorrectly_after_no_explanation=[], [], [], []
    
    for idx, (user_id, content_id , part) in test_df[['user_id', 'content_id','part']].iterrows():
        # check if user exists
        if user_id in state:
            # check if user already answered the question, if so update it to a maximum of 4
            if content_id in state[user_id]['user_content_attempts']:
                state[user_id]['user_content_attempts'][content_id] = min(4, state[user_id]['user_content_attempts'][content_id] + 1)
            # if user did not answered the question already, set the number of attempts to 0
            else:
                state[user_id]['user_content_attempts'][content_id] = 0
        
        # else create user with default values
        else:
            dict_keys = ['mean_user_accuracy', 'answered_correctly_user', 'answered_user','answered_incorrectly_user','user_residual','lecture_seen','user_content_attempts',
                         'nbr_question_task','prior_question_nbr','mean_prior_elapsed_time','all_prior_had_explanation','mean_prior_had_explanation',
                         'mean_user_part_accuracy', 'answered_correctly_user_part','answered_user_part','answered_incorrectly_user_part','user_residual_part','lecture_part_seen',
                         'answered_correctly_after_explanation','answered_incorrectly_after_explanation','answered_correctly_after_no_explanation','answered_incorrectly_after_no_explanation']
            
            dict_default_vals = [0.680, 0, 0, 0, 0, 0,dict(zip([content_id],[0])),0,0,0,0,0,dict(zip([part],[0.680])),
                                 dict(zip([part],[0])),dict(zip([part],[0])),dict(zip([part],[0])),dict(zip([part],[0])),dict(zip([part],[0])),
                                 0,0,0,0]
            state[user_id] = dict(zip(dict_keys, dict_default_vals))
            
        if not part in state[user_id]['mean_user_part_accuracy']:
            state[user_id]['mean_user_part_accuracy'][part] = 0.680
            state[user_id]['answered_correctly_user_part'][part] = 0
            state[user_id]['answered_user_part'][part] = 0
            state[user_id]['answered_incorrectly_user_part'][part] = 0
            state[user_id]['user_residual_part'][part] = 0
            
        if not part in state[user_id]['lecture_part_seen']:
                state[user_id]['lecture_part_seen'][part] = 0
        # if user did not answered the question already, set the number of attempts to 0
        #else:
        #    state[user_id]['user_content_attempts'][content_id] = 0
            
        # add user data to lists
        attempt.append(state[user_id]['user_content_attempts'][content_id])
        mean_user_accuracy.append(state[user_id]['mean_user_accuracy'])
        answered_correctly_user.append(state[user_id]['answered_correctly_user'])
        answered_user.append(state[user_id]['answered_user'])
        answered_incorrectly_user.append(state[user_id]['answered_incorrectly_user'])
        user_residual.append(state[user_id]['user_residual'])
        lecture_seen.append(state[user_id]['lecture_seen'])
        
        answered_correctly_after_explanation.append(state[user_id]['answered_correctly_after_explanation'])
        answered_incorrectly_after_explanation.append(state[user_id]['answered_incorrectly_after_explanation'])
        answered_correctly_after_no_explanation.append(state[user_id]['answered_correctly_after_no_explanation'])
        answered_incorrectly_after_no_explanation.append(state[user_id]['answered_incorrectly_after_no_explanation'])
        
        if state[user_id]['mean_prior_elapsed_time'] == 0:
            mean_prior_elapsed_time.append(23916)
        else:
            mean_prior_elapsed_time.append(state[user_id]['mean_prior_elapsed_time'])
        all_prior_had_explanation.append(state[user_id]['all_prior_had_explanation'])
        mean_prior_had_explanation.append(state[user_id]['mean_prior_had_explanation'])
        
        mean_user_part_accuracy.append(state[user_id]['mean_user_part_accuracy'][part])
        answered_correctly_user_part.append(state[user_id]['answered_correctly_user_part'][part])
        answered_user_part.append(state[user_id]['answered_user_part'][part])
        answered_incorrectly_user_part.append(state[user_id]['answered_incorrectly_user_part'][part])
        user_residual_part.append(state[user_id]['user_residual_part'][part])
        lecture_part_seen.append(state[user_id]['lecture_part_seen'][part])
    
    return attempt, mean_user_accuracy, answered_correctly_user, answered_user,answered_incorrectly_user,user_residual,lecture_seen,mean_prior_elapsed_time,all_prior_had_explanation,mean_prior_had_explanation,mean_user_part_accuracy, answered_correctly_user_part, answered_user_part,answered_incorrectly_user_part,user_residual_part,lecture_part_seen,answered_correctly_after_explanation,answered_incorrectly_after_explanation,answered_correctly_after_no_explanation,answered_incorrectly_after_no_explanation

In [9]:
# updates the user data
def update_user_data(state, features_questions_df, prev_test_df):
    for user_id, content_id, answered_correctly,part,prior_question_had_explanation,question_accuracy in prev_test_df[['user_id', 'content_id', 'answered_correctly','part','prior_question_had_explanation','question_accuracy']].values:
        # update user features
        
        state[user_id]['user_residual'] = ((state[user_id]['user_residual']*state[user_id]['answered_user'])+(answered_correctly-question_accuracy))/(state[user_id]['answered_user']+1)
        
        state[user_id]['answered_correctly_user'] += answered_correctly
        state[user_id]['answered_user'] += 1
        state[user_id]['mean_user_accuracy'] = state[user_id]['answered_correctly_user'] / state[user_id]['answered_user']
        state[user_id]['answered_incorrectly_user'] += (1-answered_correctly)
        
        
        if prior_question_had_explanation:
            
            if answered_correctly ==1 :
                state[user_id]['answered_correctly_after_explanation'] += 1
            else:
                state[user_id]['answered_incorrectly_after_explanation'] += 1
        
        else:
            if answered_correctly ==1 :
                state[user_id]['answered_correctly_after_no_explanation'] += 1
            else:
                state[user_id]['answered_incorrectly_after_no_explanation'] += 1 
        
        # update user part features
        state[user_id]['user_residual_part'][part] = (((state[user_id]['user_residual_part'][part])*(state[user_id]['answered_user_part'][part]))+
                                                      (answered_correctly-question_accuracy))/(state[user_id]['answered_user_part'][part]+1)
        
        state[user_id]['answered_correctly_user_part'][part] += answered_correctly
        state[user_id]['answered_user_part'][part] += 1
        state[user_id]['mean_user_part_accuracy'][part] = state[user_id]['answered_correctly_user_part'][part] / state[user_id]['answered_user_part'][part]
        state[user_id]['answered_incorrectly_user_part'][part] += (1-answered_correctly)

In [10]:
#update prior data
def update_user_prior_data(state, test_df):
    grouped=test_df.groupby(['user_id','task_container_id'],as_index=False).agg({'content_id':'count','prior_question_elapsed_time':'first','prior_question_had_explanation':'first'})
    grouped.columns=['user_id','task_container_id','question_count','prior_question_elapsed_time','prior_question_had_explanation']
    for user_id, task_container_id, question_count,prior_question_elapsed_time,prior_question_had_explanation in grouped.values:
        # update user features
        if not user_id in state:
            dict_keys = ['mean_user_accuracy', 'answered_correctly_user', 'answered_user','answered_incorrectly_user','user_residual','lecture_seen','user_content_attempts',
                         'nbr_question_task','prior_question_nbr','mean_prior_elapsed_time','all_prior_had_explanation','mean_prior_had_explanation',
                         'mean_user_part_accuracy', 'answered_correctly_user_part','answered_user_part','answered_incorrectly_user_part','user_residual_part','lecture_part_seen',
                         'answered_correctly_after_explanation','answered_incorrectly_after_explanation','answered_correctly_after_no_explanation','answered_incorrectly_after_no_explanation']
            dict_default_vals = [0.680, 0, 0, 0, 0, 0,dict(),question_count,0,0,0,0,dict(),
                                 dict(),dict(),dict(),dict(),dict(),
                                 0,0,0,0]
            state[user_id] = dict(zip(dict_keys, dict_default_vals))
            
        else:   
            state[user_id]['mean_prior_elapsed_time'] = ((state[user_id]['mean_prior_elapsed_time'] *state[user_id]['prior_question_nbr'])+(prior_question_elapsed_time*state[user_id]['nbr_question_task']))/(state[user_id]['prior_question_nbr']+state[user_id]['nbr_question_task'])
            if prior_question_had_explanation:
                state[user_id]['all_prior_had_explanation'] += state[user_id]['nbr_question_task']
            state[user_id]['prior_question_nbr']+=state[user_id]['nbr_question_task']  
            state[user_id]['mean_prior_had_explanation'] = state[user_id]['all_prior_had_explanation']/state[user_id]['prior_question_nbr']
            state[user_id]['nbr_question_task']=question_count
            
        
            

# Make actual prediction

In [11]:
import riiideducation

env = riiideducation.make_env()
iter_test = env.iter_test()

During prediction the previous test_df is saved and updated using the prior_group_answers_correct field of the next test_df.

This data is than used to update the user features in the state.

In [12]:
#model=lgb.Booster(model_file='../input/data-merged/model_V2_ans_expl_time_resid_ids_2500.lgb')
model=lgb.Booster(model_file='../input/data-merged/model_V0_lag_time_lec.lgb')

In [13]:
prev_test_df = None
#mean_attempt_acc_factor = FEATURES['mean_attempt_acc_factor']

for idx, (test_df, _) in tqdm(enumerate(iter_test)):
    # from 2nd iteration, update user data
    print(test_df.dtypes)
    if prev_test_df is not None:
        prev_test_df['answered_correctly'] = eval(test_df['prior_group_answers_correct'].iloc[0])
        update_user_data(state, features_questions_df, prev_test_df.loc[prev_test_df['content_type_id'] == 0])
        if idx is 1:
            display(test_df)
            display(prev_test_df)
        
    
    # merge with all features
    test_df = features_questions_df.merge(test_df, how='right', right_on='content_id',left_on='question_id')
    
    #update prior data using the test data
    update_user_prior_data(state, test_df.loc[test_df['content_type_id'] == 0])
    
    # get user data from state and update attempt
    attempt, mean_user_accuracy, answered_correctly_user, answered_user,answered_incorrectly_user,user_residual,lecture_seen,mean_prior_elapsed_time,all_prior_had_explanation,mean_prior_had_explanation,mean_user_part_accuracy, answered_correctly_user_part, answered_user_part,answered_incorrectly_user_part,user_residual_part,lecture_part_seen,answered_correctly_after_explanation,answered_incorrectly_after_explanation,answered_correctly_after_no_explanation,answered_incorrectly_after_no_explanation = get_user_data(state, test_df)

    # set updated user data
    test_df['attempt'] = attempt
    test_df['user_correctness'] = mean_user_accuracy
    test_df['answered_correctly_user'] = answered_correctly_user
    test_df['answered_user'] = answered_user
    test_df['answered_incorrectly_user'] = answered_incorrectly_user
    test_df['user_residual'] = user_residual
    test_df['lecture_seen'] = lecture_seen
    test_df['mean_prior_elapsed_time'] = mean_prior_elapsed_time
    test_df['all_prior_had_explanation'] = all_prior_had_explanation
    test_df['mean_prior_had_explanation'] = mean_prior_had_explanation
    test_df['user_part_correctness']=mean_user_part_accuracy
    test_df['answered_correctly_user_part']=answered_correctly_user_part
    test_df['answered_user_part']=answered_user_part
    test_df['answered_incorrectly_user_part']=answered_incorrectly_user_part
    test_df['user_residual_part']=user_residual_part
    test_df['lecture_part_seen']=lecture_part_seen
    
    
    question_len=len( test_df[test_df['content_type_id'] == 0])
    delta_prior_question_elapsed_time = np.zeros(question_len, dtype=np.int32)
    lagtime = np.zeros(question_len, dtype=np.float32)
    lagtime2 = np.zeros(question_len, dtype=np.float32)
    lagtime3 = np.zeros(question_len, dtype=np.float32)
    
    
    i=0
    for j, (user_id,prior_question_had_explanation,content_type_id,prior_question_elapsed_time,timestamp, content_id,task_container_id) in enumerate(zip(test_df['user_id'].values,test_df['prior_question_had_explanation'].values,test_df['content_type_id'].values,test_df['prior_question_elapsed_time'].values,test_df['timestamp'].values, test_df['content_id'].values, test_df['task_container_id'].values)):
        try:
            if(content_type_id==1):#
                x=1
#             if(len(user_lecture_stats_part[user_lecture_stats_part.user_id==user_id])==0):
#                 user_lecture_stats_part = user_lecture_stats_part.append([{'user_id':user_id}], ignore_index=True)
#                 user_lecture_stats_part.fillna(0, inplace=True)
#                 user_lecture_stats_part.loc[user_lecture_stats_part.user_id==user_id,part_lectures_columns + types_of_lectures_columns]+=lectures_df[lectures_df.lecture_id==content_id][part_lectures_columns + types_of_lectures_columns].values
#             else:
#                 user_lecture_stats_part.loc[user_lecture_stats_part.user_id==user_id,part_lectures_columns + types_of_lectures_columns]+=lectures_df[lectures_df.lecture_id==content_id][part_lectures_columns + types_of_lectures_columns].values
            if(content_type_id==0):#   
                if user_id in max_timestamp_u_dict.keys():
                    lagtime[i]=timestamp-max_timestamp_u_dict[user_id]
                    if(max_timestamp_u_dict2[user_id]==lagtime_mean2):#
                        lagtime2[i]=lagtime_mean2
                        lagtime3[i]=lagtime_mean3
                        #max_timestamp_u_dict3['max_time_stamp3'].update({user_id:lagtime_mean3})
                    else:
                        lagtime2[i]=timestamp-max_timestamp_u_dict2[user_id]
                        if(max_timestamp_u_dict3[user_id]==lagtime_mean3):
                            lagtime3[i]=lagtime_mean3 #
                        else:
                            lagtime3[i]=timestamp-max_timestamp_u_dict3[user_id]
                    
                        max_timestamp_u_dict3[user_id]=max_timestamp_u_dict2[user_id]
                        
                    max_timestamp_u_dict2[user_id]=max_timestamp_u_dict[user_id]
                    max_timestamp_u_dict[user_id]=timestamp
#                 lagtime_means[i]=(lagtime_mean_dict[user_id]+lagtime[i])/2
#                 lagtime_mean_dict[user_id]=lagtime_means[i]
                else:
                    lagtime[i]=lagtime_mean
                    max_timestamp_u_dict.update({user_id:timestamp})
                    lagtime2[i]=lagtime_mean2#
                    max_timestamp_u_dict2.update({user_id:lagtime_mean2})
                    lagtime3[i]=lagtime_mean3#
                    max_timestamp_u_dict3.update({user_id:lagtime_mean3})
#                 lagtime_mean_dict.update({user_id:lagtime_mean})
#                 lagtime_means[i]=(lagtime_mean_dict[user_id]+lagtime[i])/2
        except:
            lagtime[i]=lagtime_mean
            lagtime2[i]=lagtime_mean2#
            lagtime3[i]=lagtime_mean3#
        try:
            if user_id in user_prior_question_elapsed_time_dict.keys():            
                delta_prior_question_elapsed_time[i]=prior_question_elapsed_time-user_prior_question_elapsed_time_dict[user_id]
                user_prior_question_elapsed_time_dict[user_id]=prior_question_elapsed_time
            else:           
                delta_prior_question_elapsed_time[i]=delta_prior_question_elapsed_time_mean    
                user_prior_question_elapsed_time_dict.update({user_id:prior_question_elapsed_time})
        except:
            delta_prior_question_elapsed_time[i]=delta_prior_question_elapsed_time_mean
                
        i=i+1 

    
    
    
    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop=True)
    #
    
    test_df['lagtime'] = lagtime
    test_df['lagtime2'] = lagtime2
    test_df['lagtime3'] = lagtime3
    test_df['delta_prior_question_elapsed_time'] = delta_prior_question_elapsed_time
    
    
    test_df['lagtime']=test_df['lagtime']/(1000*3600)
    test_df.lagtime=test_df.lagtime.astype('float32')
    test_df['lagtime2']=test_df['lagtime2']/(1000*3600)
    test_df.lagtime2=test_df.lagtime2.astype('float32')
    test_df['lagtime3']=test_df['lagtime3']/(1000*3600)
    test_df.lagtime3=test_df.lagtime3.astype('float32')
    
    # set mix features answered * explanation
    
    test_df['answered_correctly_after_explanation'] = answered_correctly_after_explanation
    test_df['answered_incorrectly_after_explanation'] = answered_incorrectly_after_explanation
    test_df['answered_correctly_after_no_explanation'] = answered_correctly_after_no_explanation
    test_df['answered_incorrectly_after_no_explanation'] = answered_incorrectly_after_no_explanation
    

    
    # fill prior question had explenation
    test_df['prior_question_had_explanation'].fillna(False, inplace=True)
    test_df['prior_question_elapsed_time'].fillna(23916, inplace=True)
    test_df['prior_question_had_explanation']=test_df['prior_question_had_explanation'].astype(np.float32)

    # add harmonic mean
    test_df['hmean_user_content_accuracy'] = 2 * (
        (test_df['user_correctness'] * test_df['question_accuracy']) /
        (test_df['user_correctness'] + test_df['question_accuracy'])
    )
    
    test_df['hmean_user_part_accuracy'] = 2 * (
        (test_df['user_part_correctness'] * test_df['h_part_accuracy']) /
        (test_df['user_part_correctness'] + test_df['h_part_accuracy'])
    )
    
    test_df['h_prior_elapsed_time'] = 2 * (
        (test_df['mean_prior_elapsed_time'] * test_df['avg_question_elapsed_time']) /
        (test_df['mean_prior_elapsed_time'] + test_df['avg_question_elapsed_time'])
    )
    
    test_df['h_prior_had_explanation'] = 2 * (
        (test_df['mean_prior_had_explanation'] * test_df['avg_question_had_exlanation']) /
        (test_df['mean_prior_had_explanation'] + test_df['avg_question_had_exlanation'])
    )
    

    test_df['answered_correctly'] = model.predict(test_df[features])
    print(test_df.dtypes)

    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

    # set previour test_df
    prev_test_df = test_df.copy()

row_id                              int64
timestamp                           int64
user_id                             int32
content_id                          int16
content_type_id                      int8
task_container_id                   int16
prior_question_elapsed_time       float32
prior_question_had_explanation    boolean
prior_group_answers_correct        object
prior_group_responses              object
dtype: object
question_id                                    int16
part                                            int8
tags                                          string
tags_label                                     int16
question_accuracy                            float64
question_nbr_seen                            float64
avg_question_elapsed_time                    float32
avg_question_had_exlanation                  float64
part_accuracy                                float64
h_part_accuracy                              float64
row_id                              

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,prior_group_answers_correct,prior_group_responses
group_num,,,,,,,,,,
1,18,23609,275030867,5502,0,1,34000.0,False,"[0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, ...","[0, 0, 1, 1, 0, 1, 3, 3, 1, 1, 0, 3, 1, 2, 2, ..."
1,19,2035159380,1233875513,1512,0,1431,25000.0,True,NaN,NaN
1,20,2035159380,1233875513,1511,0,1431,25000.0,True,NaN,NaN
1,21,2035159380,1233875513,1513,0,1431,25000.0,True,NaN,NaN
1,22,1217231,891955351,9145,0,20,23000.0,True,NaN,NaN
1,23,11265012636,1981166446,3299,0,950,36333.0,True,NaN,NaN
1,24,11265012636,1981166446,3297,0,950,36333.0,True,NaN,NaN
1,25,11265012636,1981166446,3298,0,950,36333.0,True,NaN,NaN
1,26,4693145319,1637273633,11373,0,3148,41000.0,True,NaN,NaN


,question_id,part,tags,tags_label,question_accuracy,question_nbr_seen,avg_question_elapsed_time,avg_question_had_exlanation,part_accuracy,h_part_accuracy,...,delta_prior_question_elapsed_time,answered_correctly_after_explanation,answered_incorrectly_after_explanation,answered_correctly_after_no_explanation,answered_incorrectly_after_no_explanation,hmean_user_content_accuracy,hmean_user_part_accuracy,h_prior_elapsed_time,h_prior_had_explanation,answered_correctly
0,5729,5,8,1218,0.559251,0.356358,21182.304688,0.745413,0.610088,0.583564,...,-8,0.0,0.0,0,0,0.613743,0.628102,22466.299007,0.000000,0
1,12010,2,90 100 92 29,1391,0.830091,0.210055,17755.208984,0.991569,0.708694,0.764604,...,1000,3636.0,1492.0,51,20,0.764884,0.777016,18615.645644,0.989336,1
2,457,2,143 105 38 29,493,0.838630,2.096058,16043.404297,0.985832,0.708694,0.768206,...,1000,133.0,109.0,4,7,0.658082,0.637595,19465.421560,0.968915,1
3,13262,5,96,1405,0.824263,0.120148,23628.472656,0.976190,0.610088,0.701185,...,-15000,175.0,67.0,12,19,0.748196,0.713027,22013.051938,0.931165,0
4,6119,5,54,991,0.446381,8.198424,23224.431641,0.983966,0.610088,0.515551,...,11150,153.0,113.0,9,8,0.501611,0.482159,30798.124799,0.966959,0
5,12023,2,2 107 92 29,902,0.544554,0.206377,18033.156250,0.989439,0.708694,0.615875,...,-5000,4718.0,1264.0,189,37,0.644850,0.711459,21664.676343,0.977255,1
6,574,2,143 141 38 29,537,0.734281,1.735340,18306.500000,0.987362,0.708694,0.721260,...,-2000,1130.0,870.0,4,8,0.637729,0.618445,22801.947045,0.991674,0
7,12043,2,143 114 38 29,504,0.829026,0.205560,20181.455078,0.990722,0.708694,0.764152,...,4000,2926.0,2356.0,14,5,0.664607,0.668114,19095.551978,0.993842,1
8,7910,1,10 178 92,17,0.630739,5.073204,22321.402344,0.975941,0.745032,0.683138,...,2000,547.0,182.0,10,5,0.684658,0.689818,26470.815464,0.977216,0
9,7908,1,131 93 38,333,0.612414,2.444788,21020.662109,0.952248,0.745032,0.672245,...,-70000,322.0,275.0,2,6,0.571402,0.661346,24449.824733,0.970002,1


question_id                                    int16
part                                            int8
tags                                          string
tags_label                                     int16
question_accuracy                            float64
question_nbr_seen                            float64
avg_question_elapsed_time                    float32
avg_question_had_exlanation                  float64
part_accuracy                                float64
h_part_accuracy                              float64
row_id                                         int64
timestamp                                      int64
user_id                                        int32
content_id                                     int16
content_type_id                                 int8
task_container_id                              int16
prior_question_elapsed_time                  float32
prior_question_had_explanation               float32
prior_group_answers_correct                   